# Generate Frames

This is useful to generate YOLOv3 Darknet bounding boxes.

In [ ]:
import cv2
import numpy as np
import pandas as pd
import os
import time

from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import matplotlib.pyplot as plt

region_data = pd.read_csv('regions.csv')
region_data

In [ ]:
cctv_data = pd.read_csv('cctv_views.csv')
cctv_data

In [ ]:
# Define a CCTV path of image whose region has been detected
# Define the map path as well.

# You can freely change this depending on the condition of your database.
region_name = "SIMPANG 4 PATUNG KUDA"
cctv_range = {'start':0, 'end': 100}
cctv_time = '1200'

region_cctv_views = cctv_data[cctv_data['region_name'] == region_name]['cctv_view_name'].values.tolist()

region_map_path = region_data[region_data['cctv_folder_parent'] == region_name]
region_map_path = 'map/' + region_map_path['map_image_name'].values.tolist()[0]

region_coords = pd.read_csv('coords/cctv/{:}.csv'.format(region_name))

print("CCTV VIEWS :", region_cctv_views)
print("MAP PATH   :", region_map_path)
print("REGION COORDINATES:")
region_coords

In [ ]:
coco_vehicle_labels = ['bicycle', 'car', 'motorbike', 'bus', 'truck']

def eliminate_non_vehicle(series):
    detections = series['object_probs'].split(' ')
    if detections[0] in coco_vehicle_labels:
        return detections[0]
    return None

ord_q = ord('q') # For quit button
ord_p = ord('p') # For pause button

def pause_and_is_finished():
    # Pauses and quit button purposes
    k = cv2.waitKey(25) & 0xFF
    finished = k == ord_q
    if k == ord_p:
        while True:
            k = cv2.waitKey(25) & 0xFF
            finished = k == ord_q
            if k == ord_p or k == ord_q:
                break
    return finished

def generate_equal_rgb_color_distance(n):
    colors = []
    for i in range(n):
        color = np.uint8(cv2.cvtColor(np.float32([[[i*360/n, 0.8, 0.9]]]), cv2.COLOR_HSV2RGB)[0, 0] * 255)
        colors.append(color)
    return colors

coco_colors = generate_equal_rgb_color_distance(len(coco_vehicle_labels))
coco_colors

def get_color_from_class(row):
    class_name = row.split()[0]
    class_index = coco_vehicle_labels.index(class_name)
    return coco_colors[class_index]

In [5]:
# Let's define a dictionary to store variables.
# These variables include precomputed value as well, such as
# the transformation matrix.

objs = {}
for i in range(len(region_cctv_views)):
    obj_dict = {}
    
    rctv = region_cctv_views[i]
    path_cctv_view = './frame/cctv_{:}_{:} {:}_x264.mp4/'.format(region_name, rctv, cctv_time)

    # Get detections CSV file name length
    # It is assumed that all files has the same name length.
    file_ext_first_file = os.path.splitext(os.listdir(path_cctv_view)[0])
    filename_length = len(file_ext_first_file[0])
    
    # Populate variables to the object dictionary.    
    obj_dict['rctv'] = rctv
    obj_dict['path_cctv_view'] = path_cctv_view
    obj_dict['filename_length'] = filename_length
    objs[i] = obj_dict

In [16]:
BOX_OUTLINE_SIZE = 5

force_quit = False
cctv_range = {'start':90, 'end': 100}
output_path_parent = './frame_annotate/'

if not os.path.exists(output_path_parent):
    os.makedirs(output_path_parent)

csv_detections_path = './detections.csv'
csv_outfile = open(csv_detections_path, 'w')
csv_outfile.writelines(','.join(coco_vehicle_labels))

FONT_SIZE    = 0.8
FONT_COLOR   = (0,0,0)
FONT_OUTLINE = 1

try:
    for i in range(cctv_range['start'], cctv_range['end'] + 1):
        # Initiate vehicle counter
        vehicle_count = np.int32(np.zeros(len(coco_vehicle_labels)))
        
        # Use only first cctv
        j = 0
        obj = objs[j]

        filename_length = obj['filename_length']
        path_cctv_view = obj['path_cctv_view']

        csv_file_name = str(i).zfill(filename_length)
        csv_path = '{:}/{:}.csv'.format(path_cctv_view, csv_file_name)\

        # Read CCTV image, replacing CSV path with JPG path
        cctv_view_image = cv2.imread(csv_path[:-3] + 'jpg')

        # Read CSV
        objects = pd.read_csv(csv_path)

        n_detection = objects.shape[0]
        if (n_detection > 0):
            # Eliminate non-vehicles object
            vehicle_labels = objects.apply(lambda r: eliminate_non_vehicle(r), axis=1)
            objects = objects[~vehicle_labels.isna()]
            vehicle_labels = vehicle_labels[~vehicle_labels.isna()]

            # Count the vehicles
            vehicle_count += [(vehicle_labels == l).sum() for l in coco_vehicle_labels]

        n_detection = objects.shape[0]
        if (n_detection > 0):
            # For every bounding box vehicles, find the bounding box center of mass
            centers = objects.apply(lambda r: [
                              (r['left'] + r['right'])/2,
                              (r['top'] + r['bottom'])/2
                              ], axis=1)

            # Get colors
            colors = objects['object_probs'].apply(lambda row: get_color_from_class(row)).values.tolist()

            # Draw bounding boxes
            boxes = objects.apply(lambda r: [
                              [r['left'], r['top']],
                              [r['right'], r['bottom']]
                              ], axis=1)
            
            # Resize the boxes
            boxes = np.int32(np.int32([k for k in boxes.values]))
            for k, box in enumerate(boxes):
                cv2.rectangle(cctv_view_image, tuple(box[0]), tuple(box[1]), tuple(colors[k].tolist()),
                              BOX_OUTLINE_SIZE)
        
        cv2.putText(cctv_view_image, 'Frame: {:}'.format(i), (40, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, FONT_SIZE, FONT_COLOR,
                    FONT_OUTLINE, cv2.LINE_AA)
        cv2.putText(cctv_view_image, 'Detected vehicles: {:}'.format(np.sum(vehicle_count)), (40, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, FONT_SIZE, FONT_COLOR,
                    FONT_OUTLINE, cv2.LINE_AA)
        
        cv2.imshow('Image CCTV', cctv_view_image)
        WRITE_PATH = output_path_parent + '/' + csv_file_name + '.jpg'
        cv2.imwrite(WRITE_PATH, cctv_view_image)
        csv_outfile.writelines('\n' + ','.join([str(i) for i in vehicle_count.tolist()]))
        
        if pause_and_is_finished():
            force_quit = True
            break
    
    if not force_quit:
        cv2.putText(cctv_view_image, 'FINISHED!'.format(np.sum(vehicle_count)), (40, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, FONT_SIZE, (255, 255, 255),
                    2, cv2.LINE_AA)
        cv2.imshow('Image CCTV', cctv_view_image)
        while not pause_and_is_finished():
            time.sleep(0.1)
finally:
    cv2.destroyAllWindows()
    csv_outfile.close()